## U-Net model

This notebook considers training and visualization of the U-Net model.

In [ ]:
try: # Google Colab integration
  from google.colab import drive

  print('Colab environment detected. Mounting drive...')
  drive.mount('/content/drive')

  print('Mounted. Switching to directory... ', end = '')
  %cd /content/drive/'My Drive'/CILroadseg
  print('done.')
except:
  print('Colab environment not found. Working on ordinary directory.')

Colab environment detected. Mounting drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted. Switching to directory... /content/drive/My Drive/CILroadseg
done.


In [ ]:
import numpy as np
np.random.seed(18)

import tensorflow as tf
tf.random.set_seed(33)

import sys
import os
import matplotlib.image as mpimg

from util.submit import *      # util/submit.py contains the functions used to generate the CSV file for Kaggle Competition
from util.visualize import *   # util/visualize.py provides functions for image visualization
from util.notebooks import *   # util/notebooks.py contains various util functions used in notebooks

## Loading Training Data

`nb_load_data` is an helper function provided in `util/notebooks.py`

In [ ]:
train_dir = "training/images/"
gt_dir = "training/groundtruth/"
test_dir = "test/images/"

X, Y, X_test = nb_load_data(train_dir, gt_dir, test_dir)

Y = (Y >= 0.25) * 1

Loading training input...
Progress: done (100 images).
Loading training groundtruth...
Progress: done (100 images).
Loading test input...
Progress: done (94 images).

       Training data shape: (100, 400, 400, 3)
Training groundtruth shape: (100, 400, 400)
           Test data shape: (94, 608, 608, 3)


## Working with the Model

The `UNetModel` class is a subclass of `ModelBase`.
This base class can be found in `util/model_base.py` and provides a common interface to all the models we created.

In particular:
- `initialize()` resets the state of the object and should be called before the training starts
- `train(Y, X)` takes the training data `X` and its groundtruth `Y` to train the model
- `classify(X)` returns the predictions for `X`
- `load(filename)` and `save(filename)` load and save weights of the Neural Network from file.

---

The UNet masks returned by `UNetModel` still have predictions in [0, 1], thus we use the `Discretizer` decorator to round them.

In [ ]:
from tensorflow import keras

from discretize import *
from rotate_mean import *
from unet import *

Using TensorFlow backend.


In [ ]:
model = UNetModel()
# For the U-Net model, data augmentation and
# recomposition are done in the UNetModel class.

model_dsc = Discretizer(model, threshold=0.5)
# We round the labels to 0 or 1, using 0.5 as threshold.

model_rnm = Discretizer(RotAndMean(model), threshold=0.5)
# Segments the image for every possible rotation/flip
# combinations and computes the pixelwise mean of predictions
# then the mean is rounded to the nearest integer as in model_dsc.


DO_TRAINING = False
if DO_TRAINING:
  model.initialize()
  model.train(Y, X)

  model.save("saves/final/unet.h5")
else:
  model.initialize()
  model.load("saves/final/unet-rotation+color.h5")

seed: 4
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 304, 304, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 304, 304, 32) 2432        input_1[0][0]                    
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 304, 304, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 304, 304, 32) 25632       re_lu_1[0][0]                    
____________________________________________________________________________________

## Making Predictions with the Model

`UNetModel.classify(X)` takes the array of images and directly returns the array of full masks.

`Discretizer.classify(X)` rounds the values of the output regardless of its shape.

The function `nb_predict_masks` is an helper function provided in `util/notebooks.py`, while `masks_to_submission` is a function based on the implementation provided in the Kaggle competition.

The following two cells can be skipped if you do not want to generate the `.csv` file.

In [ ]:
test_masks_dir = "test/pred/unet/"
test_dir = "test/images/"

# replace model_rnm with model_dsc if you want to generate
# the csv file without rotate and mean.
nb_predict_masks(model_rnm, test_dir, test_masks_dir)

Predicting test cases... 
Progress: done.


In [ ]:
image_paths = [test_masks_dir + file for file in os.listdir(test_masks_dir)]
masks_to_submission("test/unet.csv", image_paths)

This prediction achieved an F1 score of 0.916 in Kaggle's public test set.

# Visualizing predictions

The function `view_image_array` is provided in `util/visualize.py`. It uses `matplotlib` to visualize the images and the corresponding predictions.

In [ ]:
Y_pred     = model.classify(X_test[0:10])
Y_pred_dsc = model_dsc.classify(X_test[0:10])
Y_pred_rnm = model_rnm.classify(X_test[0:10])

view_image_array(X_test[0:10], Y_pred, Y_pred_dsc, Y_pred_rnm)

# For each row we have: image, unrounded mask, rounded mask, mask with rotate and mean